In [1]:
import numpy as np
import pandas as pd

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

glove.6B.100d.txt.zip
glove6b100dtxt
sample_submission.csv
sample_submission.csv.zip
test.csv
test.csv.zip
train.csv
train.csv.zip



Using TensorFlow backend.


## Loading data

In [2]:
max_features = 20000
maxlen = 100

train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
train_df = train_df.sample(frac=1)

list_sentences_train = train_df["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_df[list_classes].values
list_sentences_test = test_df["comment_text"].fillna("CVxTz").values

## Model

In [3]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)


In [8]:
X_train[:3]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,   161,    98,     2,    64,     6,   643,
          536,    15,     1,   254,   106,  5956,    18,     6, 15190,
          215],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

In [4]:
def get_model(num_cell):
    embed_size = 300
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Bidirectional(LSTM(num_cell, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.2)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

## Training

In [5]:
model = get_model(32)
batch_size = 64
epochs = 10


file_path="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=20)

callbacks_list = [checkpoint, early] #early
model.fit(X_train, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143613/143613 [==============================] - 536s - loss: 0.0706 - acc: 0.9768 - val_loss: 0.0478 - val_acc: 0.9824
Epoch 2/10
143613/143613 [==============================] - 532s - loss: 0.0463 - acc: 0.9830 - val_loss: 0.0459 - val_acc: 0.9828
Epoch 3/10
143613/143613 [==============================] - 530s - loss: 0.0399 - acc: 0.9846 - val_loss: 0.0459 - val_acc: 0.9830
Epoch 4/10
143613/143613 [==============================] - 532s - loss: 0.0343 - acc: 0.9864 - val_loss: 0.0484 - val_acc: 0.9830
Epoch 5/10
143613/143613 [==============================] - 533s - loss: 0.0302 - acc: 0.9879 - val_loss: 0.0522 - val_acc: 0.9818
Epoch 6/10
143613/143613 [==============================] - 532s - loss: 0.0263 - acc: 0.9894 - val_loss: 0.0578 - val_acc: 0.9823
Epoch 7/10
143613/143613 [==============================] - 532s - loss: 0.0233 - acc: 0.9906 - val_loss: 0.0638 - val_acc: 0.9825
Epoch 8/10
143613/143613 [======

In [6]:
batch_size = 128
epochs = 5


file_path="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=20)

callbacks_list = [checkpoint, early] #early
model.fit(X_train, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 274s - loss: 0.0131 - acc: 0.9947 - val_loss: 0.0912 - val_acc: 0.9813
Epoch 2/5
143613/143613 [==============================] - 269s - loss: 0.0117 - acc: 0.9953 - val_loss: 0.0937 - val_acc: 0.9805
Epoch 3/5
143613/143613 [==============================] - 268s - loss: 0.0107 - acc: 0.9958 - val_loss: 0.0995 - val_acc: 0.9802
Epoch 4/5
143613/143613 [==============================] - 268s - loss: 0.0102 - acc: 0.9960 - val_loss: 0.1011 - val_acc: 0.9807
Epoch 5/5
143613/143613 [==============================] - 267s - loss: 0.0097 - acc: 0.9962 - val_loss: 0.1048 - val_acc: 0.9806


## Submission

In [7]:
model.load_weights(file_path)

y_test = model.predict(X_test)

In [8]:
sample_submission = pd.read_csv("../input/sample_submission.csv")

sample_submission[list_classes] = y_test

sample_submission.to_csv("../output/2_keras_baseline222.csv", index=False)